Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import spacy
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Loading Datasets and Embeddings

In [5]:
# loading adobe dataset
data=pd.read_csv("/kaggle/input/bst-300k/behaviour_simulation_train.csv")

In [45]:
# loading adobe dataset embeddings
input_directory = "/kaggle/input/embeddings-adobe-bertweet"
npy_files = glob.glob(os.path.join(input_directory, '*.npy'))
npy_files.sort()
dfx=pd.read_csv("/kaggle/input/bst-300k/behaviour_simulation_train.csv")
arrays = [np.load(file) for file in npy_files[4:6]]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_adobe = pd.DataFrame(combined_array, columns=column_names)

In [46]:
# our test split for validation
df_adobe=df_adobe[:290000]
df_adobe['likes']=dfx['likes'][:290000]


In [47]:
df_adobe

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_759,column_760,column_761,column_762,column_763,column_764,column_765,column_766,column_767,likes
0,-0.040222,0.108009,0.160713,0.039218,0.088356,0.031966,0.077149,0.044769,0.137463,-0.106389,...,0.038848,0.034291,0.103484,0.042001,-0.166122,0.090028,-0.027626,-0.001073,-0.004191,1
1,-0.041894,0.232693,0.194009,-0.083462,0.109151,-0.072582,0.068379,-0.114513,0.033524,-0.175675,...,0.204463,-0.070391,0.020141,0.039888,-0.220657,0.120735,-0.098855,-0.027532,-0.089357,2750
2,-0.083084,0.199032,0.215147,-0.097050,0.052297,-0.069919,0.052429,-0.097068,0.025941,-0.220989,...,0.221214,-0.017033,0.056050,0.092622,-0.228663,0.139946,-0.138513,-0.048965,-0.097601,57
3,-0.061937,0.111225,0.109293,0.043520,0.108027,0.069807,0.072331,0.027757,0.121424,-0.107128,...,0.103736,0.015231,0.089077,0.073860,-0.168217,0.117653,-0.041586,0.022041,-0.069738,152
4,-0.027357,0.009223,0.115176,0.085549,0.098298,0.114739,0.128228,0.100759,0.304607,-0.075766,...,-0.035693,0.175309,0.234405,0.225582,-0.348211,0.101782,-0.077303,0.079507,-0.150786,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289995,-0.057798,0.215980,0.202596,-0.090488,0.089499,-0.075285,0.063809,-0.098405,0.033970,-0.197755,...,0.206142,-0.022094,0.050538,0.060910,-0.227439,0.141023,-0.111615,-0.040737,-0.101007,1023
289996,-0.105177,0.174510,0.210308,-0.059720,0.097524,0.228565,-0.226542,0.133556,0.219483,-0.603352,...,0.100055,0.318272,-0.209285,0.338648,-0.516608,0.217676,0.017303,0.041506,-0.302776,1566
289997,-0.151791,0.237341,0.193900,-0.010815,-0.117595,0.142661,-0.089353,0.100854,0.147450,-0.464550,...,0.108622,0.062852,0.018123,0.220504,-0.545987,0.217433,-0.051675,0.025104,-0.290967,1050
289998,-0.087534,0.132228,0.101462,0.129073,0.095613,0.104103,0.069768,0.132809,0.126198,-0.054663,...,-0.057867,0.104340,0.061973,0.089554,-0.261432,-0.007584,-0.006149,0.035313,0.000916,735


In [54]:
# loading embeddings of 20 most popular people

input_directory = "/kaggle/input/all-embeddings"
npy_files = glob.glob(os.path.join(input_directory, '*.npy'))
npy_files.sort()
arrays = [np.load(file) for file in npy_files[0:1] ]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_addon = pd.DataFrame(combined_array, columns=column_names)


arrays = [np.load(file) for file in npy_files[5:6] ]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed


df_addon_likes = pd.DataFrame({"likes":combined_array})

df_addon['likes']=df_addon_likes['likes']

In [48]:
# loading embeddings of the hugging face dataset

input_directory = "/kaggle/input/huggingf-tweets"
npy_files = glob.glob(os.path.join(input_directory, '*.npy'))
npy_files.sort()


arrays = [np.load(file) for file in npy_files[0:1] ]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
column_names = [f"column_{i}" for i in range(combined_array.shape[1])]
df_hugf = pd.DataFrame(combined_array, columns=column_names)



arrays = [np.load(file,allow_pickle=True) for file in npy_files[2:4] ]
combined_array = np.concatenate(arrays, axis=0)  # Adjust the axis as needed

# Convert the combined array to a Pandas DataFrame
df_hugf_likes = pd.DataFrame({"likes":combined_array})

df_hugf['likes']=df_hugf_likes["likes"]

In [55]:
df_hugf
df_adobe
df_addon

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_759,column_760,column_761,column_762,column_763,column_764,column_765,column_766,column_767,likes
0,-0.154127,-0.151344,-0.377448,-0.077330,0.095869,0.411940,-0.069944,0.289084,0.114994,0.096856,...,-0.012054,0.177247,0.031935,0.401372,-0.228197,-0.038693,-0.004862,-0.053549,0.127753,7900
1,-0.167472,0.193662,0.130381,-0.010914,0.118921,-0.031526,0.090654,-0.001853,0.120182,-0.094159,...,0.048626,0.019898,0.027002,0.082022,-0.072014,0.080217,-0.048844,0.043605,-0.091369,3689
2,0.091672,0.114496,0.235802,-0.045015,0.224426,0.461116,0.071483,-0.028599,0.158961,-0.104609,...,-0.005488,-0.029499,0.017324,0.123109,-0.122801,0.167925,0.010058,0.267849,-0.068502,10341
3,-0.162463,-0.051926,0.265765,-0.096245,0.156943,0.081077,0.249380,-0.042717,0.168813,-0.257451,...,0.288909,0.063993,0.104457,0.023103,-0.115189,0.128620,0.050526,0.259585,-0.058861,10774
4,-0.088413,0.105983,0.146393,0.021852,0.172199,0.146673,-0.014989,0.006294,0.027295,-0.043853,...,0.089705,0.017486,0.088891,0.098165,-0.187963,0.059195,-0.076993,-0.043770,0.021594,17620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52537,-0.045161,0.216978,0.223599,-0.111616,0.109365,-0.099478,0.064633,-0.106406,0.068605,-0.161105,...,0.203380,-0.002450,0.046334,0.073337,-0.176256,0.123473,-0.076478,-0.025770,-0.092236,32799
52538,-0.061771,0.199032,0.216818,-0.064656,0.027647,0.010387,0.015847,-0.063588,0.055930,-0.269613,...,0.243148,0.015492,0.096675,0.105014,-0.288742,0.159232,-0.119379,-0.006264,-0.133784,21709
52539,-0.043759,0.150520,0.107826,-0.020806,0.137003,-0.015146,0.040299,0.014591,0.092801,-0.102785,...,0.078205,0.005475,0.014508,0.122956,-0.127841,0.152805,-0.074619,-0.024388,-0.016255,25269
52540,0.220253,-0.051640,0.313727,-0.144343,-0.052349,0.235636,-0.110695,-0.069330,0.221120,-0.381823,...,-0.062351,0.241984,-0.148425,0.210350,-0.439895,0.304517,-0.030173,0.301381,-0.316979,15985


In [56]:
# combining the embeddings
final_df=pd.concat([df_adobe, df_addon, df_hugf], ignore_index=True)
final_df

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_759,column_760,column_761,column_762,column_763,column_764,column_765,column_766,column_767,likes
0,-0.040222,0.108009,0.160713,0.039218,0.088356,0.031966,0.077149,0.044769,0.137463,-0.106389,...,0.038848,0.034291,0.103484,0.042001,-0.166122,0.090028,-0.027626,-0.001073,-0.004191,1
1,-0.041894,0.232693,0.194009,-0.083462,0.109151,-0.072582,0.068379,-0.114513,0.033524,-0.175675,...,0.204463,-0.070391,0.020141,0.039888,-0.220657,0.120735,-0.098855,-0.027532,-0.089357,2750
2,-0.083084,0.199032,0.215147,-0.097050,0.052297,-0.069919,0.052429,-0.097068,0.025941,-0.220989,...,0.221214,-0.017033,0.056050,0.092622,-0.228663,0.139946,-0.138513,-0.048965,-0.097601,57
3,-0.061937,0.111225,0.109293,0.043520,0.108027,0.069807,0.072331,0.027757,0.121424,-0.107128,...,0.103736,0.015231,0.089077,0.073860,-0.168217,0.117653,-0.041586,0.022041,-0.069738,152
4,-0.027357,0.009223,0.115176,0.085549,0.098298,0.114739,0.128228,0.100759,0.304607,-0.075766,...,-0.035693,0.175309,0.234405,0.225582,-0.348211,0.101782,-0.077303,0.079507,-0.150786,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463000,-0.062508,-0.000219,0.104526,0.149094,0.077520,0.340422,0.096860,0.079741,0.151080,0.012302,...,-0.039842,0.116987,0.114974,0.135970,-0.022117,0.046944,-0.060451,0.045211,-0.029745,8944
463001,-0.062508,-0.000219,0.104526,0.149094,0.077520,0.340422,0.096860,0.079741,0.151080,0.012302,...,-0.039842,0.116987,0.114974,0.135970,-0.022117,0.046944,-0.060451,0.045211,-0.029745,8820
463002,-0.062508,-0.000219,0.104526,0.149094,0.077520,0.340422,0.096860,0.079741,0.151080,0.012302,...,-0.039842,0.116987,0.114974,0.135970,-0.022117,0.046944,-0.060451,0.045211,-0.029745,12349
463003,-0.062508,-0.000219,0.104526,0.149094,0.077520,0.340422,0.096860,0.079741,0.151080,0.012302,...,-0.039842,0.116987,0.114974,0.135970,-0.022117,0.046944,-0.060451,0.045211,-0.029745,20452


In [5]:
# Assuming your data is stored in a CSV file
data_path = '/kaggle/input/bst-300k/behaviour_simulation_train.csv'
df = pd.read_csv(data_path)

In [ ]:
data_path = '/kaggle/input/bst-300k/behaviour_simulation_train.csv'
# df = pd.read_csv(data_path)
# df2=df
index1=0
# index2=463004
index2=1000
df=final_df[index1:index2]

# Function to create classes based on the number of likes
def create_likes_classes(likes):
    if likes <= 100:
        return '0-100'
    elif 100 < likes <= 1000:
        return '100-1000'
    elif 1000 < likes <= 5000:
        return '1000-5000'
    elif 5000 < likes <= 10000:
        return '5000-10000'
    elif 10000 < likes <= 100000:
        return '10000-100000'
    else:
        return '100000-1000000'

# Create a new column 'Likes_Class' with classes based on the number of likes
df['Likes_Class'] = df['likes'].apply(create_likes_classes)

y_classification = df['Likes_Class']

In [67]:
# creating the test data
X_combined3=df.drop("Likes_Class",axis=1)
X_combined3


,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,...,column_759,column_760,column_761,column_762,column_763,column_764,column_765,column_766,column_767,likes
0,-0.040222,0.108009,0.160713,0.039218,0.088356,0.031966,0.077149,0.044769,0.137463,-0.106389,...,0.038848,0.034291,0.103484,0.042001,-0.166122,0.090028,-0.027626,-0.001073,-0.004191,1
1,-0.041894,0.232693,0.194009,-0.083462,0.109151,-0.072582,0.068379,-0.114513,0.033524,-0.175675,...,0.204463,-0.070391,0.020141,0.039888,-0.220657,0.120735,-0.098855,-0.027532,-0.089357,2750
2,-0.083084,0.199032,0.215147,-0.097050,0.052297,-0.069919,0.052429,-0.097068,0.025941,-0.220989,...,0.221214,-0.017033,0.056050,0.092622,-0.228663,0.139946,-0.138513,-0.048965,-0.097601,57
3,-0.061937,0.111225,0.109293,0.043520,0.108027,0.069807,0.072331,0.027757,0.121424,-0.107128,...,0.103736,0.015231,0.089077,0.073860,-0.168217,0.117653,-0.041586,0.022041,-0.069738,152
4,-0.027357,0.009223,0.115176,0.085549,0.098298,0.114739,0.128228,0.100759,0.304607,-0.075766,...,-0.035693,0.175309,0.234405,0.225582,-0.348211,0.101782,-0.077303,0.079507,-0.150786,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.056455,0.068807,0.146569,0.041066,0.092206,0.050583,0.046584,0.041404,0.140227,-0.193869,...,0.056372,-0.002473,0.082443,0.025086,-0.171761,0.114395,-0.038208,0.070423,-0.050967,27
996,-0.072015,0.221821,0.209806,-0.104613,0.066532,-0.083490,0.048948,-0.116277,0.060162,-0.187932,...,0.218611,-0.036622,0.040717,0.076408,-0.212188,0.128792,-0.110801,-0.047272,-0.089772,492
997,-0.035416,-0.059287,-0.036267,0.081311,0.094856,0.210478,0.150726,0.219568,0.193689,-0.206943,...,0.001814,-0.071778,0.022941,0.281565,-0.233456,0.146162,-0.139829,0.002452,-0.149618,1538
998,-0.141213,0.230550,0.202926,-0.005192,-0.073810,0.068037,-0.013410,0.012459,-0.026669,-0.291240,...,0.153270,0.002083,0.085458,0.141379,-0.371550,0.151947,-0.137327,-0.017484,-0.153182,46


In [68]:
y_classification

0          0-100
1      1000-5000
2          0-100
3       100-1000
4          0-100
         ...    
995        0-100
996     100-1000
997    1000-5000
998        0-100
999        0-100
Name: Likes_Class, Length: 1000, dtype: object

In [69]:

# oversampling the data
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_combined3, y_classification)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train the classifier
classifier = RandomForestClassifier(n_estimators=200,max_depth=50,class_weight='balanced', random_state=46)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
joblib.dump(classifier, 'random_forest_model_bertTweet(290k).joblib')
print(f'Classification Accuracy: {accuracy * 100:.2f}%')

Classification Accuracy: 95.35%
